# Capstone project

For this project you'll dig into a large amount of text and apply most of what you've covered in this unit and in the course so far.

First, pick a set of texts. This can be either a series of novels, chapters, or articles. Anything you'd like. It just has to have multiple entries of varying characteristics. At least 100 should be good. There should also be at least 10 different authors, but try to keep the texts related (either all on the same topic of from the same branch of literature - something to make classification a bit more difficult than obviously different subjects).

This capstone can be an extension of your NLP challenge if you wish to use the same corpus. If you found problems with that data set that limited your analysis, however, it may be worth using what you learned to choose a new corpus. Reserve 25% of your corpus as a test set.

The first technique is to create a series of clusters. Try several techniques and pick the one you think best represents your data. Make sure there is a narrative and reasoning around why you have chosen the given clusters. Are authors consistently grouped into the same cluster?

Next, perform some unsupervised feature generation and selection using the techniques covered in this unit and elsewhere in the course. Using those features then build models to attempt to classify your texts by author. Try different permutations of unsupervised and supervised techniques to see which combinations have the best performance.

Lastly return to your holdout group. Does your clustering on those members perform as you'd expect? Have your clusters remained stable or changed dramatically? What about your model? Is it's performance consistent?

If there is a divergence in the relative stability of your model and your clusters, delve into why.

Your end result should be a write up of how clustering and modeling compare for classifying your texts. What are the advantages of each? Why would you want to use one over the other? Approximately 3-5 pages is a good length for your write up, and remember to include visuals to help tell your story!

In [102]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
import nltk
from collections import Counter
from sklearn.model_selection import train_test_split

In [103]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


# Load text data

In [104]:
emma=gutenberg.raw('austen-emma.txt')
kjv=gutenberg.raw('bible-kjv.txt')
busterbrown=gutenberg.raw('burgess-busterbrown.txt')
alice=gutenberg.raw('carroll-alice.txt')
ball=gutenberg.raw('chesterton-ball.txt')
parents=gutenberg.raw('edgeworth-parents.txt')
mobydick=gutenberg.raw('melville-moby_dick.txt')
paradise=gutenberg.raw('milton-paradise.txt')
caesar=gutenberg.raw('shakespeare-caesar.txt')
leaves=gutenberg.raw('whitman-leaves.txt')

In [105]:
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

In [106]:
# Cleaning each text

emma = re.sub(r'CHAPTER .*', '', emma)
emma = re.sub(r'VOLUME .*', '', emma)
emma = text_cleaner(emma[0:35000])

kjv = re.sub(r'\d+:\d+','', kjv)
kjv = text_cleaner(kjv[0:35000])

busterbrown = text_cleaner(busterbrown[0:35000])

alice = re.sub(r'CHAPTER .*','',alice)
alice = text_cleaner(alice[0:35000])

ball = text_cleaner(ball[0:35000])

parents = text_cleaner(parents[0:35000])

mobydick = text_cleaner(mobydick[0:35000])

paradise = re.sub(r'Book [IVX]','',paradise)
paradise = text_cleaner(paradise[0:35000])

caesar = text_cleaner(caesar[0:35000])

leaves = text_cleaner(leaves[0:35000])

In [107]:

nlp = spacy.load('en')

emma_doc = nlp(emma)
kjv_doc = nlp(kjv)
busterbrown_doc = nlp(busterbrown)
alice_doc = nlp(alice)
ball_doc = nlp(ball)
parents_doc = nlp(parents)
mobydick_doc = nlp(mobydick)
paradise_doc = nlp(paradise)
caesar_doc = nlp(caesar)
leaves_doc = nlp(leaves)

In [108]:
# Group into sentences.
emma_sents = [[sent, "Austen",1] for sent in emma_doc.sents]
kjv_sents = [[sent, "bible",2] for sent in kjv_doc.sents]
busterbrown_sents = [[sent, "Burgess",3] for sent in busterbrown_doc.sents]
alice_sents = [[sent, "Carroll",4] for sent in alice_doc.sents]
ball_sents = [[sent, "Cherterton",5] for sent in ball_doc.sents]
parents_sents = [[sent, "Edgeworth",6] for sent in parents_doc.sents]
mobydick_sents = [[sent, "Melville",7] for sent in mobydick_doc.sents]
paradise_sents = [[sent, "Milton",8] for sent in paradise_doc.sents]
caesar_sents = [[sent, "Shakespeare",9] for sent in caesar_doc.sents]
leaves_sents = [[sent, "Whitman",10] for sent in leaves_doc.sents]

In [109]:
print(len(emma_sents),len(kjv_sents),len(busterbrown_sents),len(alice_sents),
      len(ball_sents),len(parents_sents),len(mobydick_sents),len(paradise_sents),
      len(caesar_sents),len(leaves_sents))

265 300 423 364 340 236 406 200 593 315


In [112]:
sentences=[[str(sent) for sent in emma_doc.sents]+[str(sent) for sent in kjv_doc.sents]+
[str(sent) for sent in busterbrown_doc.sents]+[str(sent) for sent in alice_doc.sents]+
[str(sent) for sent in ball_doc.sents]+[str(sent) for sent in parents_doc.sents]+
[str(sent) for sent in mobydick_doc.sents]+[str(sent) for sent in paradise_doc.sents]+
[str(sent) for sent in caesar_doc.sents]+[str(sent) for sent in leaves_doc.sents]]

sentences=sentences[0]

author_code=([0]*len(emma_sents)+[1]*len(kjv_sents)+[2]*len(busterbrown_sents)+[3]*len(alice_sents) 
+[4]*len(ball_sents)+[5]*len(parents_sents)+[6]*len(mobydick_sents)+[7]*len(paradise_sents) 
+[8]*len(caesar_sents)+[9]*len(leaves_sents))

df=pd.DataFrame()
df['sentences']=sentences
df['author_code']=author_code

In [111]:
from sklearn.feature_extraction.text import TfidfVectorizer

X=df['sentences']
Y=df['author_code']

vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than half the paragraphs
                             min_df=2, # only use words that appear at least twice
                             stop_words='english', 
                             lowercase=True, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )

#Applying the vectorizer
paras_tfidf=vectorizer.fit_transform(X)
print("Number of features: %d" % paras_tfidf.get_shape()[1])

#splitting into training and test sets
X_train_tfidf, X_test_tfidf, Y_train, Y_test= train_test_split(paras_tfidf, Y, test_size=0.25, random_state=0)


#Reshapes the vectorizer output into something people can read
X_train_tfidf_csr = X_train_tfidf.tocsr()

#number of paragraphs
n = X_train_tfidf_csr.shape[0]
#A list of dictionaries, one per paragraph
tfidf_bypara = [{} for _ in range(0,n)]
#List of features
terms = vectorizer.get_feature_names()
#for each paragraph, lists the feature words and their tf-idf scores
for i, j in zip(*X_train_tfidf_csr.nonzero()):
    tfidf_bypara[i][terms[j]] = X_train_tfidf_csr[i, j]

#Keep in mind that the log base 2 of 1 is 0, so a tf-idf score of 0 indicates that the word was present once in that sentence.
# print('Original sentence:', X_train[5])
print('Tf_idf vector:', tfidf_bypara[5])

Number of features: 3312
Tf_idf vector: {'smarter': 0.3578218548864647, 'acquainted': 0.3915557741271254, 'smart': 0.33808877712832225, 'wasn': 0.3578218548864647, 'bit': 0.3329611170497163, 'buster': 0.45097870459732237, 'quite': 0.2794941200509133, 'know': 0.22391080014770018, 'little': 0.18387274900243136}


In [142]:
# For testing, I just used 3 books 
a=[str(sent) for sent in emma_doc.sents]
b=[str(sent) for sent in busterbrown_doc.sents]
c=[str(sent) for sent in kjv_doc.sents]
d=vectorizer.transform(a)
e=vectorizer.transform(b)
f=vectorizer.transform(c)


abc=np.concatenate((d.toarray(),e.toarray(),f.toarray()), axis=0)

Y=[0]*len(emma_sents)+[1]*len(busterbrown_sents)+[2]*len(kjv_sents)
Y=np.transpose(Y)
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans

# Normalize the data.
X_norm = normalize(abc,axis=1)

# Calculate predicted values.
y_pred = KMeans(n_clusters=3, random_state=42).fit_predict(X_norm)

# Check the solution against the data.
print('Comparing k-means clusters against authors:')
print(pd.crosstab(Y, y_pred))

# Calculating Adjusted Rand Index
from sklearn import metrics

print('ARI :')
print(metrics.adjusted_rand_score(Y, y_pred))

Comparing k-means clusters against authors:
col_0    0   1    2
row_0              
0      261   0    4
1      301   0  122
2      251  49    0
ARI :
0.010361415879410633


In [15]:
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans

# Normalize the data.
X_norm = normalize(X_train_lsa,axis=1)

# Calculate predicted values.
y_pred = KMeans(n_clusters=10, random_state=42).fit_predict(X_norm)

# Check the solution against the data.
print('Comparing k-means clusters against authors:')
print(pd.crosstab(Y_train, y_pred))

# Calculating Adjusted Rand Index
from sklearn import metrics

print('ARI :')
print(metrics.adjusted_rand_score(Y_train, y_pred))

Comparing k-means clusters against authors:
col_0         0    1    2   3   4   5   6   7   8   9
author_code                                          
0             4    1  177   0   3   3   1   0  10   0
1             8    0   66  74   0   0  39   0  21  12
2             9  129  157   0   3   1   0   0  13   0
3             8    7  194   1   8  11   0   0  33   6
4             8    2  216   6   3   1   0   0  23   3
5             6    0  150   0   0   0   0   0  18   2
6             5    1  240   3   0  50   0   0   3   8
7             3    0  138   6   1   1   0   0   1   6
8            16    0  311  18  36   0   0  39  10  13
9             2    0  193  14   0   0   0   0   3  23
ARI :
0.018450424904392915
